In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import pprint
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
# import apartment info from mongodb
db = client['apartments_com']
ind_apts_table = db['individual_apartments']

In [3]:
ind_apts_table.count_documents({})

586

In [5]:
#create blank dataframe with just column names
df_columns = ['listing_id', 'property_name', 'address', 'mean_rent', 'mean_area', 'walkscore']
df = pd.DataFrame(columns=df_columns)

In [4]:
apartment_collection = ind_apts_table.find()

In [6]:
# for each apartment in collection parse html and 
# find the following info in collection 
# [name, address, price or price range, area, and Walkscore] 
# and add to Pandas DF
for apartment in apartment_collection:
    soup = BeautifulSoup(apartment['html'], 'html.parser')
    
    listing_id = apartment['listing_id']
    property_name = soup.find('h1', 'propertyName').text.strip()
    address_ = soup.find('div', 'propertyAddress').find('h2').text
    address = ' '.join(address_.split())
    
    # iterate through each unit and find mean price and area
    max_rents = []
    unit_areas = []
    for unit in soup.find('table', 'availabilityTable').find('tbody').find_all('tr', 'rentalGridRow'):
        availability = unit.find('td', 'available').text.strip()
        # only look at max rents for simplicity
        unit_max_rent = unit['data-maxrent']
        if (availability == 'Available Now') and (unit_max_rent != ''):
            max_rents.append(int(unit_max_rent))
        else:
            break
        unit_area_str = unit.find('td', 'sqft').text.strip(' Sq Ft')
        unit_area_range = unit_area_str.replace(',', '').split(' - ')
        try:
            unit_area_range = [int(area) for area in unit_area_range]
            unit_area = (max(unit_area_range) + min(unit_area_range) / 2)
            unit_areas.append(unit_area)
        except:
            print('unit_area problem: {}'.format(address))
            
    mean_rent = 0
    mean_area = 0
    try:
        mean_rent = sum(max_rents) / len(max_rents)
        mean_area = sum(unit_areas) / len(unit_areas)
    except:
        print('problem: {}'.format(address))
    
    walkscore = int(soup.find('div', 'ratingCol walkScore').find('span', 'score').text)
    
    #add info to DF
    row = pd.DataFrame([[listing_id, 
                         property_name, 
                         address, 
                         mean_rent, 
                         mean_area, 
                         walkscore]], columns=df_columns)
    df = df.append(row, ignore_index=True)
    
    

problem: 2704 Rio Grande St, Austin, TX 78705
problem: 110 San Antonio St, Austin, TX 78701
problem: 2810 Hemphill Park, Austin, TX 78705
problem: 2815 Guadalupe St, Austin, TX 78705
problem: 13005 Heinemann Dr, Austin, TX 78727
problem: 1600 Wickersham Ln, Austin, TX 78741
problem: 1300 Crossing Pl, Austin, TX 78741
problem: 1833 Cheddar Loop, Austin, TX 78728
problem: 1301 Crossing Pl, Austin, TX 78741
problem: 1901 Mariposa Dr, Austin, TX 78741
problem: 1500 E Riverside Dr, Austin, TX 78741
problem: 5501 Ross Rd, Del Valle, TX 78617
problem: 507 W 23rd St, Austin, TX 78705
problem: 1114 Camino La Costa, Austin, TX 78752
problem: 3603 Southridge Dr, Austin, TX 78704
problem: 1801 Wells Branch Pky, Austin, TX 78728
unit_area problem: 4711 E Riverside Dr, Austin, TX 78741
problem: 2101 Rio Grande, Austin, TX 78705
problem: 400 W Anderson Ln, Austin, TX 78752
problem: 615 Wonsley Dr E, Austin, TX 78753
problem: 1016 Camino La Costa, Austin, TX 78752
problem: 1044 Camino La Costa, Austin

In [19]:
df = df[df['mean_rent'] != 0]
df.info

,listing_id,property_name,address,mean_rent,mean_area,walkscore,cost/SF
0,p9c5ygt,Dobie Twenty21 Student Spaces,"2021-2025 Guadalupe St, Austin, TX 78705",1791.500000,434.750000,95,4.120759
1,rdsyjd0,The District on La Frontera,"2811 La Frontera Blvd, Austin, TX 78728",2130.714286,926.142857,12,2.300632
2,vsvz28g,Cliffs at Barton Creek,"3050 Tamarron Blvd, Austin, TX 78746",2585.545455,1079.772727,39,2.394527
3,pfzryrs,Radius On Grove,"2301 Grove Blvd, Austin, TX 78741",1604.000000,1042.500000,34,1.538609
4,lb7xscf,Chandelier,"2336 Douglas St, Austin, TX 78741",1274.333333,697.000000,47,1.828312


In [12]:
# Create column for cost/SF

# Create column for some metric comparing cost/SF and Walkscore. 
# Maybe it's the ((cost/SF) / Walkscore)

In [13]:
df['cost/SF'] = (df['mean_rent'] / df['mean_area'])
df.sort_values(by='cost/SF')

/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,listing_id,property_name,address,mean_rent,mean_area,walkscore,cost/SF
258,6v748nv,Texan26,"1009 W 26th St, Austin, TX 78705",851.500000,1375.000000,89,0.619273
380,4h44qtj,Waters at Willow Run,"15433 FM 1325, Austin, TX 78728",1070.000000,1100.000000,2,0.972727
550,mkjrmgh,Eagles Landing,"8000 Decker Ln, Austin, TX 78724",993.666667,1009.666667,1,0.984153
584,6vvhrpr,6500 Adair Dr,"Austin, TX 78754",1650.000000,1636.000000,3,1.008557
532,j81y3r7,River Valley Apartments,"810 E Slaughter Ln, Austin, TX 78744",1240.250000,1200.250000,21,1.033326
...,...,...,...,...,...,...,...
446,j5v1qd5,Terrazzo,"8585 Spicewood Springs Rd, Austin, TX 78759",4311.875000,883.500000,52,4.880447
498,hp1ncd2,Timbercreek Apartments,"614 S 1st St, Austin, TX 78704",3975.625000,788.875000,68,5.039613
91,56q2bdm,Lenox Boardwalk,"2515 Elmont Dr, Austin, TX 78741",4916.437500,938.937500,64,5.236171
47,mdgjmyk,Mosaic at Mueller,"4600 Mueller Blvd, Austin, TX 78723",7224.680000,1124.720000,60,6.423537


In [41]:
#export to csv
df.to_csv('../data/dataframe.csv', index=False)

In [20]:
# some walkscores are objects and not int
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 489 entries, 0 to 585
Data columns (total 7 columns):
listing_id       489 non-null object
property_name    489 non-null object
address          489 non-null object
mean_rent        489 non-null float64
mean_area        489 non-null float64
walkscore        489 non-null object
cost/SF          489 non-null float64
dtypes: float64(3), object(4)
memory usage: 30.6+ KB


In [35]:
for row in range(len(df)):
    walkscore = df.iloc[row].loc['walkscore']
    if isinstance(int(), type(walkscore)) == False:
        print(df.iloc[row])

In [39]:
# df['rent/WS'] = (df['mean_rent'] / df['walkscore'])
# df['area/WS'] = (df['mean_area'] / df['walkscore'])
df[df['walkscore'] == 0]

,listing_id,property_name,address,mean_rent,mean_area,walkscore,cost/SF
15,m377rhl,The Santal,"7624 Tecoma Cir, Austin, TX 78735",1928.888889,1153.055556,0,1.672850
107,tmv7f48,Urbana at Goodnight Ranch,"9005 Alderman Dr, Austin, TX 78747",1720.555556,1029.518519,0,1.671224
